## Import Dependencies

In [1]:
import re 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html2text
import stanza
import spacy
from spacy_stanza import StanzaLanguage
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import wordninja

In [2]:
data = pd.read_csv('data - data.csv',memory_map=True)
data = data[['text']]
data['clean_text'] = data.text

In [3]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)
#remove non-ascii characters
data.clean_text = data.clean_text.map(lambda x: _removeNonAscii(x))


In [4]:
# remove special characters, numbers, punctuations
import contractions

# data['clean_text'] = data['clean_text'].apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) )
data['clean_text'] = data['clean_text'].apply(lambda x: re.sub(r'http\S+', '', x))
data['clean_text'] = data['clean_text'].apply(
    lambda x: [contractions.fix(word) for word in x.split()])


In [5]:
data['clean_text'] = data['clean_text'].apply(lambda x: ' '.join(x))

In [6]:
data['clean_text'] = data['clean_text'].str.replace("[^a-zA-Z0-9]", " ")

In [7]:
# data.clean_text = data.clean_text.apply(lambda x: [wordninja.split(word) for word in x.split()])

In [8]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [9]:
data.clean_text

0                       Just happened a terrible car crash
1        Our Deeds are the Reason of this  earthquake M...
2        Heard about  earthquake is different cities  s...
3        there is a forest fire at spot pond  geese are...
4                   Forest fire near La Ronge Sask  Canada
                               ...                        
10871           M1 94  01 04 UTC  5km S of Volcano Hawaii 
10872    Police investigating after an e bike collided ...
10873    The Latest  More Homes Razed by Northern Calif...
10874           MEG issues Hazardous Weather Outlook  HWO 
10875     CityofCalgary has activated its Municipal Eme...
Name: clean_text, Length: 10876, dtype: object

In [10]:
#turn all comments' tokens into one single list
data.clean_text  = data.clean_text.apply(lambda x: x.split())
unlist_comments = [item for items in data.clean_text for item in items]

In [11]:
unlist_comments

['Just',
 'happened',
 'a',
 'terrible',
 'car',
 'crash',
 'Our',
 'Deeds',
 'are',
 'the',
 'Reason',
 'of',
 'this',
 'earthquake',
 'May',
 'ALLAH',
 'Forgive',
 'us',
 'all',
 'Heard',
 'about',
 'earthquake',
 'is',
 'different',
 'cities',
 'stay',
 'safe',
 'everyone',
 'there',
 'is',
 'a',
 'forest',
 'fire',
 'at',
 'spot',
 'pond',
 'geese',
 'are',
 'fleeing',
 'across',
 'the',
 'street',
 'I',
 'cannot',
 'save',
 'them',
 'all',
 'Forest',
 'fire',
 'near',
 'La',
 'Ronge',
 'Sask',
 'Canada',
 'All',
 'residents',
 'asked',
 'to',
 'shelter',
 'in',
 'place',
 'are',
 'being',
 'notified',
 'by',
 'officers',
 'No',
 'other',
 'evacuation',
 'or',
 'shelter',
 'in',
 'place',
 'orders',
 'are',
 'expected',
 '13',
 '000',
 'people',
 'receive',
 'wildfires',
 'evacuation',
 'orders',
 'in',
 'California',
 'Just',
 'got',
 'sent',
 'this',
 'photo',
 'from',
 'Ruby',
 'Alaska',
 'as',
 'smoke',
 'from',
 'wildfires',
 'pours',
 'into',
 'a',
 'school',
 'RockyFire',
 '

> We will then use NLTK’s tools to generate all possible bigrams and trigrams

In [12]:
import nltk
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_comments)

## Methods to Rank Collocations

### Counting frequencies of adjacent words with part of speech filters:

In [13]:
bigram_freq = bigramFinder.ngram_fd.items()

In [14]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [15]:
bigramFreqTable.reset_index(drop=True)

,bigram,freq
0,"(I, am)",478
1,"(in, the)",366
2,"(of, the)",333
3,"(it, is)",304
4,"(do, not)",287
...,...,...
98201,"(yet, justicemalala)",1
98202,"(Ahrar, not)",1
98203,"(and, Ahrar)",1
98204,"(Nusrah, and)",1


In [16]:
bigramFreqTable[:10]

,bigram,freq
148,"(I, am)",478
161,"(in, the)",366
152,"(of, the)",333
144,"(it, is)",304
286,"(do, not)",287
29,"(is, a)",176
2652,"(to, the)",162
752,"(to, be)",162
471,"(on, the)",160
933,"(in, a)",158


``
Issue with this is adjacent spaces, stop words, articles, prepositions or pronouns are common and are not meaningful
To fix this, we filter out for collocations not containing stop words and filter for only the following structures:``
  
  - **Bigrams: (Noun, Noun), (Adjective, Noun)** \
  
  - **Trigrams: (Adjective/Noun, Anything, Adjective/Noun)** \
 
 *This is a common structure used in literature and generally works well* 

In [17]:
en_stopwords = set(stopwords.words('english'))
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [18]:
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [19]:
filtered_bi[:10]

,bigram,freq
3988,"(YouTube, video)",61
13425,"(suicide, bomber)",58
5332,"(Northern, California)",53
16965,"(burning, buildings)",50
4519,"(Full, read)",48
15787,"(mass, murder)",44
6155,"(suicide, bombing)",44
13722,"(Cross, Body)",43
83579,"(PKK, suicide)",42
83578,"(old, PKK)",41


In [20]:

trigram_freq = trigramFinder.ngram_fd.items()

In [21]:
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

In [22]:
trigramFreqTable.head().reset_index(drop=True)

,trigram,freq
0,"(I, do, not)",68
1,"(liked, a, YouTube)",61
2,"(a, YouTube, video)",61
3,"(I, liked, a)",61
4,"(I, can, not)",47


In [23]:
trigramFreqTable[:10]

,trigram,freq
294,"(I, do, not)",68
4424,"(liked, a, YouTube)",61
4425,"(a, YouTube, video)",61
4423,"(I, liked, a)",61
319,"(I, can, not)",47
106877,"(bomber, who, detonated)",42
106879,"(detonated, bomb, in)",42
106878,"(who, detonated, bomb)",42
106876,"(suicide, bomber, who)",42
106875,"(PKK, suicide, bomber)",42


In [24]:
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [25]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]

In [26]:
filtered_tri[:10]

,trigram,freq
106875,"(PKK, suicide, bomber)",42
106874,"(old, PKK, suicide)",41
99025,"(Airport, Get, Swallowed)",34
106890,"(Turkey, Army, trench)",34
126117,"(MH370, Malaysia, PM)",32
47495,"(Obama, Declares, Disaster)",30
47160,"(Knock, Detonation, Sensor)",28
47159,"(Ignition, Knock, Detonation)",28
90398,"(Refugio, oil, spill)",27
97944,"(Are, There, Gems)",26


## PMI 
-  **The main intuition is that it measures how much more likely the words co-occur than if they were independent.**

In [27]:
freq_bi = filtered_bi[:20].bigram.values

In [28]:
freq_tri = filtered_tri[:20].trigram.values

In [29]:
bigramFinder.apply_freq_filter(20)

In [30]:
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [31]:
bigramPMITable[:10]

,bigram,PMI
0,"(Madhya, Pradesh)",12.818283
1,"(Pamela, Geller)",12.818283
2,"(Ignition, Knock)",12.470359
3,"(Virgin, Galactic)",12.470359
4,"(subreddits, banned)",12.461426
5,"(Christian, Attacked)",12.455713
6,"(Temple, Mount)",12.441213
7,"(Offensive, Content)",12.363150
8,"(Work, Light)",12.285935
9,"(Quarantine, Offensive)",12.233320


In [32]:
trigramFinder.apply_freq_filter(20)

In [33]:
trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)

In [34]:
trigramPMITable[:25]

,trigram,PMI
0,"(anthrax, lab, mishaps)",24.632596
1,"(Quarantine, Offensive, Content)",24.596470
2,"(Waving, Israeli, Flag)",24.533061
3,"(Ignition, Knock, Detonation)",24.500146
4,"(horrible, subreddits, banned)",24.453738
5,"(Knock, Detonation, Sensor)",24.449520
6,"(Funtenna, hijacking, computers)",24.375519
7,"(Wreckage, Conclusively, Confirmed)",24.260195
8,"(transport, bioterror, germs)",24.073629
9,"(River, Wild, Horse)",23.818942


In [35]:
pmi_bi = bigramPMITable[:20].bigram.values
pmi_tri = trigramPMITable[:20].trigram.values

## T-test
- **T-test has been criticized as it assumes normal distribution. Therefore, we will also look into the chi-square test.**

In [36]:
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)

In [37]:
bigramTtable.head()

,bigram,t
0,"(I, am)",21.519910
1,"(it, is)",16.789338
2,"(do, not)",16.748021
3,"(in, the)",16.073035
4,"(of, the)",15.095408


In [38]:
filteredT_bi = bigramTtable[bigramTtable.bigram.map(lambda x: rightTypes(x))]

In [39]:
filteredT_bi[:10]

,bigram,t
30,"(YouTube, video)",7.795500
35,"(suicide, bomber)",7.606961
44,"(Northern, California)",7.268854
49,"(burning, buildings)",7.056831
55,"(Full, read)",6.920995
60,"(mass, murder)",6.627967
61,"(suicide, bombing)",6.622073
67,"(Cross, Body)",6.553820
70,"(PKK, suicide)",6.475081
76,"(old, PKK)",6.398089


In [40]:
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','t']).sort_values(by='t', ascending=False)

In [41]:
trigramTtable.head()

,trigram,t
0,"(I, do, not)",8.240293
1,"(liked, a, YouTube)",7.810140
2,"(a, YouTube, video)",7.810002
3,"(I, liked, a)",7.808226
4,"(I, can, not)",6.850316


In [42]:
filteredT_tri = trigramTtable[trigramTtable.trigram.map(lambda x: rightTypesTri(x))]

In [43]:
filteredT_tri.head(10)

,trigram,t
5,"(PKK, suicide, bomber)",6.480738
11,"(old, PKK, suicide)",6.403120
17,"(Airport, Get, Swallowed)",5.830951
18,"(Turkey, Army, trench)",5.830951
44,"(MH370, Malaysia, PM)",5.656851
55,"(Obama, Declares, Disaster)",5.477224
67,"(Ignition, Knock, Detonation)",5.291502
68,"(Knock, Detonation, Sensor)",5.291502
73,"(Refugio, oil, spill)",5.196152
89,"(Funtenna, hijacking, computers)",5.099019


In [44]:

t_bi = filteredT_bi[:20].bigram.values

In [45]:
t_tri = filteredT_tri[:20].trigram.values

## Chi-Square
- **The chi-square test assumes in the null hypothesis that words are independent, just like in t-test.

In [46]:
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [47]:
bigramChiTable.head(20)

,bigram,chi-sq
0,"(Ignition, Knock)",158895.000000
1,"(Madhya, Pradesh)",158895.000000
2,"(Pamela, Geller)",151671.545328
3,"(Conclusively, Confirmed)",144155.926871
4,"(Offensive, Content)",131705.635890
5,"(Virgin, Galactic)",130516.785120
6,"(subreddits, banned)",129710.841940
7,"(costlier, bigger)",126175.763761
8,"(Quarantine, Offensive)",125184.483946
9,"(Stock, Market)",124314.786052


In [48]:
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [49]:
trigramChiTable.head(20)

,trigram,chi-sq
0,"(Ignition, Knock, Detonation)",6.644111e+08
1,"(Knock, Detonation, Sensor)",6.415004e+08
2,"(Quarantine, Offensive, Content)",6.341841e+08
3,"(anthrax, lab, mishaps)",6.242546e+08
4,"(Wreckage, Conclusively, Confirmed)",6.027923e+08
5,"(Funtenna, hijacking, computers)",5.658950e+08
6,"(horrible, subreddits, banned)",5.284897e+08
7,"(Waving, Israeli, Flag)",5.098077e+08
8,"(transport, bioterror, germs)",4.237374e+08
9,"(Airport, Get, Swallowed)",3.668458e+08


In [50]:
chi_bi = bigramChiTable[:20].bigram.values

In [51]:
chi_tri = trigramChiTable[:20].trigram.values

## Likelihood

In [52]:
bigramLikTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [53]:
bigramLikTable.head()

,bigram,likelihood ratio
0,"(I, am)",3923.840593
1,"(do, not)",2305.536272
2,"(it, is)",1566.415894
3,"(can, not)",1061.803191
4,"(going, to)",1035.233462


In [54]:

filteredLik_bi = bigramLikTable[bigramLikTable.bigram.map(lambda x: rightTypes(x))]

In [55]:
filteredLik_bi.head(10)

,bigram,likelihood ratio
7,"(suicide, bomber)",754.289677
8,"(YouTube, video)",709.281722
13,"(Northern, California)",644.891626
14,"(Cross, Body)",642.957271
17,"(Full, read)",632.992075
19,"(loud, bang)",624.695035
20,"(mass, murder)",612.350282
21,"(oil, spill)",596.703267
24,"(PKK, suicide)",585.394848
25,"(old, PKK)",579.265825


In [56]:
trigramLikTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.likelihood_ratio)), columns=['trigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [57]:
trigramLikTable.head()

,trigram,likelihood ratio
0,"(I, am, not)",6869.440972
1,"(I, am, a)",6270.927880
2,"(I, am, going)",6170.728109
3,"(and, I, am)",6056.168759
4,"(I, am, so)",6049.362832


In [58]:
filteredLik_tri = trigramLikTable[trigramLikTable.trigram.map(lambda x: rightTypesTri(x))]

In [59]:
filteredLik_tri.head(20)

,trigram,likelihood ratio
13,"(PKK, suicide, bomber)",2145.188416
24,"(old, PKK, suicide)",1786.769136
26,"(Airport, Get, Swallowed)",1759.738765
29,"(Turkey, Army, trench)",1694.202621
35,"(Refugio, oil, spill)",1610.549650
38,"(Cross, Body, Bag)",1569.344790
40,"(Ignition, Knock, Detonation)",1554.440561
43,"(Knock, Detonation, Sensor)",1541.391192
52,"(Northern, California, Wildfire)",1474.798483
53,"(MH370, Malaysia, PM)",1456.635142


In [60]:

lik_bi = filteredLik_bi[:20].bigram.values

In [61]:
lik_tri = filteredLik_tri[:20].trigram.values

## Different Approach Comparison

In [62]:
bigramsCompare = pd.DataFrame([freq_bi, pmi_bi, t_bi, chi_bi, lik_bi]).T

In [63]:
bigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test', 'Likeihood Ratio Test With Filter']

In [64]:
bigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test,Likeihood Ratio Test With Filter
0,"(YouTube, video)","(Madhya, Pradesh)","(YouTube, video)","(Ignition, Knock)","(suicide, bomber)"
1,"(suicide, bomber)","(Pamela, Geller)","(suicide, bomber)","(Madhya, Pradesh)","(YouTube, video)"
2,"(Northern, California)","(Ignition, Knock)","(Northern, California)","(Pamela, Geller)","(Northern, California)"
3,"(burning, buildings)","(Virgin, Galactic)","(burning, buildings)","(Conclusively, Confirmed)","(Cross, Body)"
4,"(Full, read)","(subreddits, banned)","(Full, read)","(Offensive, Content)","(Full, read)"
5,"(mass, murder)","(Christian, Attacked)","(mass, murder)","(Virgin, Galactic)","(loud, bang)"
6,"(suicide, bombing)","(Temple, Mount)","(suicide, bombing)","(subreddits, banned)","(mass, murder)"
7,"(Cross, Body)","(Offensive, Content)","(Cross, Body)","(costlier, bigger)","(oil, spill)"
8,"(PKK, suicide)","(Work, Light)","(PKK, suicide)","(Quarantine, Offensive)","(PKK, suicide)"
9,"(old, PKK)","(Quarantine, Offensive)","(old, PKK)","(Stock, Market)","(old, PKK)"


In [65]:
trigramsCompare = pd.DataFrame([freq_tri, pmi_tri, t_tri, chi_tri, lik_tri]).T

In [66]:
trigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test', 'Likelihood Ratio Test With Filter']

In [67]:
trigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test,Likelihood Ratio Test With Filter
0,"(PKK, suicide, bomber)","(anthrax, lab, mishaps)","(PKK, suicide, bomber)","(Ignition, Knock, Detonation)","(PKK, suicide, bomber)"
1,"(old, PKK, suicide)","(Quarantine, Offensive, Content)","(old, PKK, suicide)","(Knock, Detonation, Sensor)","(old, PKK, suicide)"
2,"(Airport, Get, Swallowed)","(Waving, Israeli, Flag)","(Airport, Get, Swallowed)","(Quarantine, Offensive, Content)","(Airport, Get, Swallowed)"
3,"(Turkey, Army, trench)","(Ignition, Knock, Detonation)","(Turkey, Army, trench)","(anthrax, lab, mishaps)","(Turkey, Army, trench)"
4,"(MH370, Malaysia, PM)","(horrible, subreddits, banned)","(MH370, Malaysia, PM)","(Wreckage, Conclusively, Confirmed)","(Refugio, oil, spill)"
5,"(Obama, Declares, Disaster)","(Knock, Detonation, Sensor)","(Obama, Declares, Disaster)","(Funtenna, hijacking, computers)","(Cross, Body, Bag)"
6,"(Knock, Detonation, Sensor)","(Funtenna, hijacking, computers)","(Ignition, Knock, Detonation)","(horrible, subreddits, banned)","(Ignition, Knock, Detonation)"
7,"(Ignition, Knock, Detonation)","(Wreckage, Conclusively, Confirmed)","(Knock, Detonation, Sensor)","(Waving, Israeli, Flag)","(Knock, Detonation, Sensor)"
8,"(Refugio, oil, spill)","(transport, bioterror, germs)","(Refugio, oil, spill)","(transport, bioterror, germs)","(Northern, California, Wildfire)"
9,"(Are, There, Gems)","(River, Wild, Horse)","(Funtenna, hijacking, computers)","(Airport, Get, Swallowed)","(MH370, Malaysia, PM)"


- We can see that PMI and chi-square methods give pretty good results even without applying filters.
- Frequency and T-test methods are also similar to each other. 
- In real applications, we can eyeball the list and set a threshold at a value from when the list stops making sense.

In [68]:
COL_ORDER = ['PMI', 'Chi-Sq Test','Likelihood Ratio Test With Filter', 'T-test With Filter', 'Frequency With Filter']
from ordered_set import OrderedSet
trigram_set = OrderedSet()
for col in COL_ORDER:
    [trigram_set.add(x) for x in trigramsCompare[col].values]

In [69]:
[' '.join(x) for x in list(trigram_set)][:20]

['anthrax lab mishaps',
 'Quarantine Offensive Content',
 'Waving Israeli Flag',
 'Ignition Knock Detonation',
 'horrible subreddits banned',
 'Knock Detonation Sensor',
 'Funtenna hijacking computers',
 'Wreckage Conclusively Confirmed',
 'transport bioterror germs',
 'River Wild Horse',
 'Salt River Wild',
 'Stock Market Crash',
 'Airport Get Swallowed',
 'Typhoon Devastated Saipan',
 'Refugio oil spill',
 'Severe Thunderstorm Warning',
 'waves Black Hat',
 'content policy goes',
 'Now Quarantine Offensive',
 'hot Funtenna hijacking']

## LDA

In [70]:
from gensim import models,corpora
import spacy
import gensim

nlp = spacy.load('en_core_web_lg')

In [71]:
data.clean_text = data.clean_text.apply(lambda x: ' '.join(x)) 

In [72]:
data

,text,clean_text
0,Just happened a terrible car crash,Just happened a terrible car crash
1,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...
2,"Heard about #earthquake is different cities, s...",Heard about earthquake is different cities sta...
3,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...
4,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask Canada
...,...,...
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,M1 94 01 04 UTC 5km S of Volcano Hawaii
10872,Police investigating after an e-bike collided ...,Police investigating after an e bike collided ...
10873,The Latest: More Homes Razed by Northern Calif...,The Latest More Homes Razed by Northern Califo...
10874,MEG issues Hazardous Weather Outlook (HWO) htt...,MEG issues Hazardous Weather Outlook HWO


In [73]:
def clean_up(text):
    removal=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
    text_out = []
    doc= nlp(text)
    for token in doc:
        if token.is_stop == False and token.is_alpha and len(token)>2 and token.pos_ not in removal:
            lemma = token.lemma_
            text_out.append(lemma)
    return text_out
    
datalist = data.text.apply(lambda x:clean_up(x))

In [77]:
dictionary = corpora.Dictionary(datalist)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in datalist]

In [78]:
num_topics =12
Lda = models.LdaMulticore
lda= Lda(doc_term_matrix, num_topics=num_topics,id2word = dictionary, 
         passes=20,chunksize=2000,random_state=3)

## Optimize # of k topics

In [79]:
coherence = []
for k in range(1,20):
    print('Round: '+str(k))
    Lda = models.LdaMulticore
    ldamodel = Lda(doc_term_matrix, num_topics=k, id2word = dictionary, passes=20, iterations=100,  chunksize = 10000, eval_every = 10)
    cm = gensim.models.coherencemodel.CoherenceModel(model=ldamodel, texts=datalist, dictionary=dictionary, coherence='c_v')
    coherence.append((k,cm.get_coherence()))

Round: 1
Round: 2
Round: 3
Round: 4
Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12
Round: 13
Round: 14
Round: 15
Round: 16
Round: 17
Round: 18
Round: 19


In [80]:
x_val = [x[0] for x in coherence]
y_val = [x[1] for x in coherence]

In [81]:
import matplotlib.pyplot as plt 
plt.plot(x_val,y_val)
plt.scatter(x_val,y_val)
plt.title('Number of Topics vs. Coherence')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence')
plt.xticks(x_val)
plt.show()

<Figure size 640x480 with 1 Axes>

In [82]:
num_topics = 8
Lda = models.LdaMulticore
lda = Lda(doc_term_matrix,
          num_topics=8,
          id2word=dictionary,
          passes=20,
          iterations=100,
          chunksize=10000,
          eval_every=10)

In [83]:
# To show initial topics
ldamodel.show_topics(20, num_words=3, formatted=False)

[(0, [('read', 0.013302596), ('Army', 0.009794609), ('israeli', 0.009666585)]),
 (1,
  [('confirm', 0.012292456),
   ('wreckage', 0.0101112425),
   ('Malaysia', 0.009023538)]),
 (2, [('loud', 0.012273228), ('ass', 0.012213165), ('crush', 0.010745451)]),
 (3,
  [('like', 0.018674383), ('disaster', 0.015766406), ('Obama', 0.014025611)]),
 (4, [('see', 0.010353183), ('year', 0.009291382), ('tragedy', 0.0088978475)]),
 (5, [('get', 0.016214984), ('people', 0.0097998455), ('like', 0.00931898)]),
 (6,
  [('wildfire', 0.014643733),
   ('California', 0.014075281),
   ('police', 0.010233409)]),
 (7, [('stop', 0.0125497), ('National', 0.010227656), ('fall', 0.008733486)]),
 (8, [('fire', 0.044150207), ('building', 0.02659707), ('burn', 0.024684789)]),
 (9,
  [('scream', 0.021004042), ('suicide', 0.014936313), ('life', 0.012863278)]),
 (10,
  [('family', 0.018695224),
   ('body', 0.01565952),
   ('Legionnaires', 0.014448645)]),
 (11, [('fear', 0.017749893), ('kill', 0.01704158), ('drown', 0.01064

In [84]:
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
topic_data =  pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, mds = 'tsne')
pyLDAvis.display(topic_data)

In [85]:

all_topics = {}
lambd = 0.5  # Adjust this accordingly
for i in range(1,8): #Adjust this to reflect number of topics chosen for final LDA model
    topic = topic_data.topic_info[topic_data.topic_info.Category == 'Topic'+str(i)]
    topic['relevance'] = topic['loglift']*(1-lambd)+topic['logprob']*lambd
    all_topics['Topic '+str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:20].values

/Users/subir/pythonenv/default/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [86]:
all_topics['Topic 1']

array(['Obama', 'disaster', 'send', 'hot', 'declare', 'devastate',
       'sound', 'transport', 'Saipan', 'wake', 'hijacking', 'computer',
       'prebreak', 'FedEx', 'war', 'bioterror', 'datum', 'mishap', 'Iran',
       'anthrax'], dtype=object)

In [87]:
pd.DataFrame(all_topics).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Topic 1,Obama,disaster,send,hot,declare,devastate,sound,transport,Saipan,wake,hijacking,computer,prebreak,FedEx,war,bioterror,datum,mishap,Iran,anthrax
Topic 2,video,content,quarantine,ban,policy,Emergency,emergency,Reddit,subreddit,rescuer,search,new,playlist,Plan,go,horrible,effect,hundred,add,flame
Topic 3,old,PKK,detonate,pic,Turkey,bomb,storm,charge,trench,boy,violent,release,fire,bomber,mph,hail,suicide,Army,weather,manslaughter
Topic 4,fire,building,burn,thunder,wanna,wild,lightning,bush,Airlines,threaten,hurricane,Browser,desire,come,cool,record,State,remove,set,Linkury
Topic 5,Hiroshima,survivor,pay,atomic,pile,mark,Japan,likely,truth,shape,anniversary,stand,CNN,carry,colour,bombing,invoice,bomb,deluge,rise
Topic 6,Legionnaires,family,debris,outbreak,body,bag,affect,sue,Reunion,spill,find,oil,Body,Bag,bagging,Quarantine,project,disea,Offensive,Island
Topic 7,derail,minute,Sandstorm,airport,mudslide,train,swallow,evacuation,hailstorm,watch,order,Roosevelt,abandon,GBBO,yyc,health,rail,morning,town,India
